In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
import time

warnings.filterwarnings("ignore")

# Reading the dataset
dataset1 = pd.read_csv("wdbc.data", sep=",").values

# Extracting the maximum value of the features which is being considered for best feature combination
dataset = dataset1[:,22:32]


#Forming a data frame 
attributes = ["ID","Diagnosis","mean_radius","mean_texture","mean_perimeter","mean_area","mean_smoothness","mean_compactness","mean_concavity","mean_concave_points","mean_symmetry","mean_fractal_dimension","se_radius","se_texture","se_perimeter","se_area","se_smoothness","se_compactness","se_concavity","se_concave_points","se_symmetry","se_fractal_dimension",
            "worst_radius","worst_texture","worst_perimeter","worst_area","worst_smoothness","worst_compactness","worst_concavity","worst_concave_points","worst_symmetry","worst_fractal_dimension"]
data = pd.DataFrame(data=dataset1,columns=attributes)
data.set_index("ID", inplace=True)

# Displaying the diagnosis(M = malignant, B = benign) and the 30 real-valued attributes
display(data.head())

,Diagnosis,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,...,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
ID,,,,,,,,,,,,,,,,,,,,,
842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.186,0.275,0.08902
84300903,M,19.69,21.25,130.0,1203.0,0.1096,0.1599,0.1974,0.1279,0.2069,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.243,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.1425,0.2839,0.2414,0.1052,0.2597,...,14.91,26.5,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.173
84358402,M,20.29,14.34,135.1,1297.0,0.1003,0.1328,0.198,0.1043,0.1809,...,22.54,16.67,152.2,1575.0,0.1374,0.205,0.4,0.1625,0.2364,0.07678
843786,M,12.45,15.7,82.57,477.1,0.1278,0.17,0.1578,0.08089,0.2087,...,15.47,23.75,103.4,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.1244


In [2]:
# Label-1 indicates malignant and Label-0 indicates benign 
data["Diagnosis"] = data["Diagnosis"].replace("M",1)
data["Diagnosis"] = data["Diagnosis"].replace("B",0)
data = data.apply(pd.to_numeric, errors="ignore")

# X contains only the first 10 features i.e only the mean values of the attributes not their standard error or maximum values

# 2 features 

# worst_radius
# worst_compactness
x1 = dataset[:,0].reshape(-1,1).astype("float64")
x2 = dataset[:,5].reshape(-1,1).astype("float64")

X1 = np.concatenate((x1,x2),axis=1)

labels = data["Diagnosis"].values.reshape(-1,1)
data.drop("Diagnosis", axis = 1, inplace = True)
final_data1 = np.concatenate([X1,labels],axis=1)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report

def evaluate(final_data):
    report = np.empty([4,6])
    m,n = final_data.shape
    elapsedTime = 0
    for i in range(5):
        np.random.seed(0)
        np.random.shuffle(final_data)
        x_train = final_data[0:int(0.8*m),0:n-1]
        y_train = final_data[0:int(0.8*m),n-1]
        x_test  = final_data[int(0.8*m):,0:n-1]
        y_test  = final_data[int(0.8*m):,n-1]
        logisticRegr = LogisticRegression()
        
        start_time = time.time()
        logisticRegr.fit(x_train,y_train)
        predictions = logisticRegr.predict(x_test)
        stop_time = time.time()
        elapsedTime = elapsedTime + (stop_time - start_time)
        
        report[0,i] = classification_report(y_test,predictions,output_dict = True).get('accuracy')
        report[1,i] = classification_report(y_test,predictions,output_dict = True).get('macro avg').get('precision')
        report[2,i] = classification_report(y_test,predictions,output_dict = True).get('macro avg').get('recall')
        report[3,i] = classification_report(y_test,predictions,output_dict = True).get('macro avg').get('f1-score')
        cm = metrics.confusion_matrix(y_test, predictions)
        print("Confusion matrix:\n",cm)
        
    for j in range(4):
        report[j,5] = np.max(report[j,0:5])
    columns1 = ['Validation 1','Validation 2','Validation 3','Validation 4','Validation 5','Maximum']
    index1 = ['Accuracy','Precision','Recall','F1-Score']
    table = pd.DataFrame(data = report,columns=columns1,index=index1)
    display(table)
    print("Time: "+str(elapsedTime/5)+"s") # To print the average time required for training and prediction

In [4]:
# 2 features
evaluate(final_data1)

Confusion matrix:
 [[73  2]
 [ 4 35]]
Confusion matrix:
 [[65  4]
 [ 8 37]]
Confusion matrix:
 [[62  7]
 [ 6 39]]
Confusion matrix:
 [[70  5]
 [ 4 35]]
Confusion matrix:
 [[65  3]
 [ 6 40]]


,Validation 1,Validation 2,Validation 3,Validation 4,Validation 5,Maximum
Accuracy,0.947368,0.894737,0.885965,0.921053,0.921053,0.947368
Precision,0.946999,0.896425,0.879795,0.910473,0.922863,0.946999
Recall,0.935385,0.882126,0.882609,0.915385,0.912724,0.935385
F1-Score,0.940789,0.887979,0.881126,0.912837,0.917064,0.940789


Time: 0.06629786491394044s
